In [7]:
import xlrd
import lxml
import io
from lxml import etree
from pathlib import Path

In [8]:
from xnf2edx import consts
from xnf2edx.utils import get_sheet_from_row, get_sheet, get_all_row_values
from xnf2edx.xnf2edx import parseShowAnswer, fixhtmlentities, parse_html_cell
from xnf2edx.consts import (
    CDATOSGENERALESSHEET, CDATOSGENERALESROW, CDATOSGENERALESINFOPOS, CDATOSGENERALESDURATIONPOS,
    CDATOSGENERALESABOUTPOS, CDATOSGENERALESPREREQUISITESPOS, CUNIDADSHEET, CUNIDADROW, CUNIDADCHAPTERNAMECOL
)

In [3]:
XNF_PATH = Path(".").absolute().joinpath("data", "input", "MOOC_RES4CITY_20230228.xlsm")

In [4]:
# Global variables
path = ""
problemSetID = 1
stufftoreturn = {"path": "", "coursename": "", "log": "", "error": ""}

# Some config to avoid unexpected behaviour
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

In [5]:
wb = xlrd.open_workbook(XNF_PATH)

### create_about_overview

In [6]:
_wb = wb

In [7]:
sheet = get_sheet(_wb, CDATOSGENERALESSHEET, CDATOSGENERALESROW)
sheetunits = get_sheet(_wb, CUNIDADSHEET, CUNIDADROW)
htmlpath = path + "/about/overview.html"

In [8]:
info = sheet.cell_value(CDATOSGENERALESINFOPOS[0], CDATOSGENERALESINFOPOS[1])
if info[:1] != "<":
    info = "<p>" + info + "</p>"
    
duration = sheet.cell_value(CDATOSGENERALESDURATIONPOS[0], CDATOSGENERALESDURATIONPOS[1])
if duration[:1] != "<":
    duration = "<p>" + duration + "</p>"

about = sheet.cell_value(CDATOSGENERALESABOUTPOS[0], CDATOSGENERALESABOUTPOS[1])
if about[:1] != "<":
    about = "<p>" + about + "</p>"
    
prerequisites = sheet.cell_value(CDATOSGENERALESPREREQUISITESPOS[0], CDATOSGENERALESPREREQUISITESPOS[1])
if prerequisites[:1] != "<":
    prerequisites = "<p>" + prerequisites + "</p>"

In [9]:
info

'<p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>\n<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>\n<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>\n<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>\n<p style="text-align:justify;">Una vez realizado est

In [10]:
duration

'<p>8 weeks</p>'

In [11]:
about

'<p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>\n<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>\n<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>\n<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>\n<p style="text-align:justify;">Una vez realizado est

In [12]:
prerequisites

'<p style="text-align:justify;">Los siguientes requisitos previos son esenciales para la realización del MC:</p><ul>\n<li>Conocimientos básicos del sistema energético.</li>\n<li>Comprensión fundamental de los mercados de la electricidad.</li></ul>'

### <span style="color:blue;">Todo OK</span>

In [15]:
sheetunits.nrows

4

In [19]:
units = ""
prev = ""
i = 1
while (
    sheetunits.nrows > i
    and sheetunits.cell_value(i, CUNIDADCHAPTERNAMECOL) != ""
):
    if prev != str(sheetunits.cell_value(i, CUNIDADCHAPTERNAMECOL)):
        units = (f"{units}<li>{sheetunits.cell_value(i, CUNIDADCHAPTERNAMECOL)}</li>")
        prev = str(sheetunits.cell_value(i, CUNIDADCHAPTERNAMECOL))
    i = i + 1
units = "<ol>" + units + "</ol>"
units

'<ol><li>Module 1: Energy communities implementation in the urban environment</li></ol>'

### <span style="color:blue;">Todo OK</span>

In [37]:
inforoot = etree.Element("section", Class="info")
info_writer = etree.ElementTree(inforoot)
inforoot.append(etree.parse(io.StringIO("<h2>Duración</h2>\n")).getroot())
inforoot.append(
    etree.parse(io.StringIO(fixhtmlentities(duration)), etree.XMLParser(recover=True, encoding="utf-8")).getroot()
)
inforoot.append(
    etree.parse(io.StringIO("<h2>Acerca de este curso</h2>\n")).getroot()
)
inforoot.append(
    etree.parse(
        io.StringIO(fixhtmlentities(info)),
        etree.XMLParser(recover=True, encoding="utf-8"),
    ).getroot()
)

print(etree.tostring(inforoot, pretty_print=True, encoding="utf-8").decode())

<section Class="info">
  <h2>Duración</h2>
  <p>8 weeks</p>
  <h2>Acerca de este curso</h2>
  <p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>
</section>



In [38]:
info

'<p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>\n<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>\n<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>\n<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>\n<p style="text-align:justify;">Una vez realizado est

### <span style="color:red;">Problema detectado!</span>
Parece que aquí se está perdiendo la información de la variable **info**. A ver si podemos concretar más

In [40]:
info

'<p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>\n<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>\n<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>\n<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>\n<p style="text-align:justify;">Una vez realizado est

In [41]:
fixhtmlentities(info)

'<p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>\n<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>\n<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>\n<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>\n<p style="text-align:justify;">Una vez realizado est

In [45]:
io.StringIO(fixhtmlentities(info))

In [43]:
a = etree.parse(io.StringIO(fixhtmlentities(info)), etree.XMLParser(recover=True, encoding="utf-8"))
print(etree.tostring(a, pretty_print=True, encoding="utf-8").decode())

<p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>



In [56]:
a = etree.parse(io.StringIO(fixhtmlentities(info)), etree.XMLParser(recover=True, encoding="utf-8"))
print(etree.tostring(a, pretty_print=True, encoding="utf-8").decode())

<p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>



In [60]:
a = etree.fromstring(info, etree.HTMLParser(recover=True, encoding="utf-8"))
print(etree.tostring(a, pretty_print=True, encoding="utf-8").decode())

<html>
  <body><p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>
<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>
<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>
<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>
<p style="text-align:justify;">Una vez rea

In [77]:
b = etree.parse(io.StringIO(fixhtmlentities(info)), etree.HTMLParser(recover=True, encoding="utf-8")).getroot()
print(etree.tostring(b, pretty_print=True, encoding="utf-8").decode())

<html>
  <body><p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>
<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>
<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>
<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>
<p style="text-align:justify;">Una vez rea

In [72]:
for it in b.iterchildren():
    print(it.tag)

body


In [95]:
elem = parse_html_cell(info)

In [96]:
print(etree.tostring(elem, pretty_print=True, encoding="utf-8").decode())

<div><p style="text-align:justify;">Este MOOC tiene como objetivo proporcionar a los alumnos una buena comprensión del concepto de comunidad energética local (CEL), su potencial para un cambio sistemático en el sistema energético y cómo ponerlas en práctica. </p>
<p style="text-align:justify;">El MOOC comenzará explicando el concepto de CEL y su relevancia en el contexto urbano. </p>
<p style="text-align:justify;">A continuación, estableceremos el marco jurídico europeo de las CEL, explicando las similitudes y diferencias entre las Comunidades Energéticas Ciudadanas y las Comunidades de Energías Renovables. </p>
<p style="text-align:justify;">A partir de aquí, exploraremos la situación actual de las comunidades energéticas en la UE y algunos casos de éxito para destacar las implicaciones para la región y el sistema energético. En concreto, queremos que los alumnos identifiquen las ventajas, los obstáculos y los retos de las CEL. </p>
<p style="text-align:justify;">Una vez realizado est

In [89]:
children = elem.getchildren()

In [90]:
len(children)

1

In [9]:
comentary = "<p>Hello!</p>"
comentary = "<solution>" + comentary + "</solution>"
elem = parse_html_cell(comentary)

In [10]:
print(etree.tostring(elem, pretty_print=True, encoding="utf-8").decode())

<div>
  <solution>
    <p>Hello!</p>
  </solution>
</div>



In [11]:
comentary = "<p>DOMO ARIGATO!</p>"
comentary = "<solution>" + comentary + "</solution>"
elem = parse_html_cell(comentary)

In [12]:
print(etree.tostring(elem, pretty_print=True, encoding="utf-8").decode())

<div>
  <solution>
    <p>DOMO ARIGATO!</p>
  </solution>
</div>

